In [ ]:
import yfinance as yf


In [ ]:
start_date = "2023-01-01" # year-month-day

In [ ]:
stocks = yf.download(['TSLA', 'GME', 'AAPL', 'NVDA'], start=start_date)
stocks = stocks.dropna()
stocks.tail(10)

[*********************100%***********************]  4 of 4 completed


Price            Close                                           High  \
Ticker            AAPL        GME        NVDA        TSLA        AAPL   
Date                                                                    
2025-03-06  235.330002  24.200001  110.559830  263.450012  237.860001   
2025-03-07  239.070007  24.000000  112.679642  262.670013  241.369995   
2025-03-10  227.479996  22.420000  106.970161  222.149994  236.160004   
2025-03-11  220.839996  22.610001  108.750000  230.580002  225.839996   
2025-03-12  216.979996  21.990000  115.739998  248.089996  221.750000   
2025-03-13  209.679993  21.950001  115.580002  240.679993  216.839996   
2025-03-14  213.490005  23.049999  121.669998  249.979996  213.949997   
2025-03-17  214.000000  23.500000  119.529999  238.009995  215.220001   
2025-03-18  212.690002  23.299999  115.430000  225.309998  215.149994   
2025-03-19  215.240005  23.709999  117.519997  235.860001  218.759995   

Price                                                 Low             \
Ticker            GME        NVDA        TSLA        AAPL        GME   
Date                                                                   
2025-03-06  25.010000  115.339389  272.649994  233.160004  24.049999   
2025-03-07  24.350000  113.469570  266.250000  234.759995  23.520000   
2025-03-10  23.580000  111.839709  253.369995  224.220001  22.260000   
2025-03-11  22.980000  112.229676  237.059998  217.449997  22.080000   
2025-03-12  23.240000  116.760002  251.839996  214.910004  21.889999   
2025-03-13  22.250000  117.760002  248.289993  208.419998  21.559999   
2025-03-14  23.150000  121.879997  251.580002  209.580002  22.290001   
2025-03-17  23.730000  122.889999  245.399994  209.970001  22.969999   
2025-03-18  23.620001  119.019997  230.100006  211.490005  23.139999   
2025-03-19  23.990000  120.449997  241.410004  213.750000  23.260000   

Price                                     Open                         \
Ticker            NVDA        TSLA        AAPL        GME        NVDA   
Date                                                                    
2025-03-06  110.209863  260.019989  234.440002  24.740000  113.519557   
2025-03-07  107.550108  250.729996  235.110001  24.000000  111.239772   
2025-03-10  105.450297  220.000000  235.539993  23.440001  109.889891   
2025-03-11  104.760361  217.020004  223.809998  22.350000  106.980159   
2025-03-12  112.879997  241.100006  220.139999  22.950001  114.120003   
2025-03-13  113.790001  232.600006  215.949997  22.049999  117.029999   
2025-03-14  118.150002  240.729996  211.250000  22.290001  118.610001   
2025-03-17  118.029999  232.800003  213.309998  23.240000  122.739998   
2025-03-18  114.540001  222.279999  214.160004  23.290001  118.000000   
2025-03-19  115.680000  229.199997  214.220001  23.400000  117.269997   

Price                     Volume                                 
Ticker            TSLA      AAPL      GME       NVDA       TSLA  
Date                                                             
2025-03-06  272.059998  45170400  3421400  321181900   98451600  
2025-03-07  259.320007  46273600  3464000  341755500  102369600  
2025-03-10  252.539993  72071200  6226900  366487400  189076900  
2025-03-11  225.309998  76137400  4226900  354865700  174896400  
2025-03-12  247.220001  62547500  3718200  323857500  142215700  
2025-03-13  248.130005  61368300  3829800  299033100  114813500  
2025-03-14  247.309998  60107600  3960500  277593500  100242300  
2025-03-17  245.059998  48073400  2899100  255501500  111900600  
2025-03-18  228.160004  42432400  2155800  299686900  111477600  
2025-03-19  231.610001  54336700  3012100  271766100  111250900

In [15]:
stocks.to_csv('stocks_data.csv')